In [1]:
import glob
import os
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import h5py
import time
import random
import torch
import torch.multiprocessing
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, Dataset, DataLoader, ConcatDataset
import torchvision
from torchvision.io import read_image
import torchvision.transforms as transforms
from torchvision.models import resnet34
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report 
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from collections import OrderedDict
mps_device = torch.device("mps")

In [2]:
csv_files = glob.glob('Data'+'/**/*.csv', recursive=True)

features = []
    
for file in csv_files:
    data = pd.read_csv(file, header=None, na_values="?", skiprows=5)
    data.columns = ["time", "avg_rss12", "var_rss12", "avg_rss13", "var_rss13", "avg_rss23", "var_rss23"] 
    data = data.drop(columns = ["time"])    
    features.append(data)

X_train = []
X_test = []
for i,f in enumerate(features):
    if len(f) != 480:
        print(np.shape(f))
        for i in range(480-len(f)):
            f = pd.concat([f, f.mean().to_frame().T])
        print(np.shape(f))
    if 'dataset1.csv' in csv_files[i] or 'dataset2.csv' in csv_files[i] or 'dataset3.csv' in csv_files[i]:
        if 'dataset3' in csv_files[i] and 'bending' in csv_files[i]:
            X_train.append(f)
        else:
            X_test.append(f)
    else:
        X_train.append(f)

(479, 6)
(480, 6)


In [3]:
Y_train = []
Y_test = []
for i,a in enumerate(csv_files):
    if 'dataset1.csv' in a or 'dataset2.csv' in a or 'dataset3.csv' in a:
        if 'dataset3' in a and 'bending' in a:
            Y_train.append(a.split('/')[1])
        else:
            Y_test.append(a.split('/')[1])
    else:
        Y_train.append(a.split('/')[1])

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
label_binarizer = LabelBinarizer().fit(Y_train)
Y_onehot_test = label_binarizer.transform(Y_test)
Y_onehot_train = label_binarizer.transform(Y_train)

In [11]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out

input_size = 6
hidden_size = 42
output_size = 7
model = SimpleRNN(input_size, hidden_size, output_size)
model.to(mps_device)

loss_func = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-04, weight_decay=1e-5)

In [12]:
train_losses, train_accs = [], []

model.eval()
for epoch in range(40):    
    for i in range(len(X_train)):
        input = torch.tensor(X_train[i].values, dtype=torch.float32).to(mps_device)
        y = torch.tensor(Y_onehot_train[i], dtype=torch.float32).to(mps_device)

        # Forward Pass
        output = model(input)
        y_pred = torch.max(output, 1)[1]
        
        train_loss = loss_func(output, label).item()
        train_acc = 100 * (y_pred == label).sum().item() / len(label)

        break
        
    train_losses.append(train_loss)
    train_accs.append(train_acc)

model.train()

RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [ ]:
Y_train[i]